<a href="https://colab.research.google.com/github/albertorock82/practicasgit/blob/main/scrapSerpSeleniumGoogleSheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gspread oauth2client selenium beautifulsoup4 webdriver-manager==3.8.4

In [ ]:
import random
import time
import unicodedata
import gspread
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from google.oauth2.service_account import Credentials
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# Configuración de Google Sheets
SHEET_NAME = "KeywordDepliadorKS_serp"  # Nombre del libro
WORKSHEET_NAME = "palabrasSerp"  # Nombre de la pestaña

def clean_text(text):
    """Elimina acentos y caracteres especiales del texto."""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8').strip()

def configure_driver():
    """Configura el navegador con Selenium."""
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    options.add_argument("start-maximized")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.6723.70 Safari/537.36"
    )
    # Remove executable_path for newer Selenium versions
    driver = webdriver.Chrome(options=options)
    return driver

def connect_to_sheet():
    """Conecta con Google Sheets usando credenciales JSON."""
    # Make sure the path to your credentials file is correct
    creds = Credentials.from_service_account_file("/content/ambient-antenna-440101-u4-d9758bb09208.json", scopes=[
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ])
    client = gspread.authorize(creds)
    return client.open(SHEET_NAME).worksheet(WORKSHEET_NAME)

def get_keywords(sheet):
    """Obtiene las palabras clave desde la columna 'keywords' y filtra las filas sin datos en la tercera columna."""
    data = sheet.get_all_values()
    if not data:
        return []

    headers = data[0]
    try:
        keyword_col_index = headers.index("keywords")
    except ValueError:
        print("Error: 'keywords' column not found in the sheet.")
        return []

    keywords_list = []
    for idx, row in enumerate(data[1:]): # Start from the second row to skip headers
        # Ensure the row has enough columns to avoid IndexError
        if len(row) > max(keyword_col_index, 2): # Check if both keyword column and the third column exist
            keyword = row[keyword_col_index].strip()
            third_col_value = row[2].strip() # Assuming the third column is still the one to check for emptiness
            if keyword and not third_col_value:
                keywords_list.append((idx + 2, keyword)) # Add 2 because we skipped header and enumerate is 0-indexed

    return keywords_list


def google_search(keyword):
    """Realiza scraping en Google para una palabra clave."""
    driver = configure_driver()
    query = f"https://www.google.com/search?q={keyword}&hl=es&gl=EC&lr=lang_es"
    driver.get(query)
    time.sleep(random.randint(10, 20))  # Pausa aleatoria para evitar bloqueos

    soup = BeautifulSoup(driver.page_source, "html.parser")
    # Adjusted to find the correct result blocks based on current Google SERP structure
    results = soup.select("div.g")

    search_results = []
    for result in results[:3]: # Limit to top 3 results
        title = result.select_one("h3")
        link = result.select_one("a")
        description = result.select_one("div[data-sncf='1']") or result.select_one("div.VwiC3b")

        title_text = title.get_text() if title else "Sin título"
        link_href = link["href"] if link else "Sin enlace"
        description_text = description.get_text() if description else "Sin descripción"

        search_results.extend([link_href, title_text, description_text])

    driver.quit()
    return search_results

def save_to_sheet(sheet, row, results):
    """Guarda los resultados (URL, Título, Descripción) en la hoja a partir de la columna C."""
    # Ensure results list has a length that is a multiple of 3 (URL, Title, Description)
    # and pad with 'N/A' if necessary to fill up to 9 columns (3 results * 3 fields)
    padded_results = results + ['N/A'] * (9 - len(results))

    sheet.update(
        range_name=f"C{row}:K{row}",
        values=[padded_results]
    )


# Conexión y ejecución
try:
    sheet = connect_to_sheet()
    keywords_data = get_keywords(sheet)

    if not keywords_data:
        print("No keywords found to process or 'keywords' column is missing.")
    else:
        print(f"Found {len(keywords_data)} keywords to process.")
        for row, keyword in keywords_data:
            print(f"Processing row {row}, keyword: {keyword}")
            try:
                results = google_search(clean_text(keyword))
                save_to_sheet(sheet, row, results)
                print(f"Successfully processed row {row}.")
            except Exception as e:
                print(f"Error processing row {row} for keyword '{keyword}': {e}")
            time.sleep(random.randint(5, 10)) # Add a small delay between processing each keyword

except Exception as e:
    print(f"An error occurred: {e}")

Found 2196 keywords to process.
Processing row 2, keyword: adss depilacion laser
Successfully processed row 2.
Processing row 3, keyword: afeitador de mujer
Successfully processed row 3.
Processing row 4, keyword: afeitador femenino


KeyboardInterrupt: 